# Movie Recommender System  using Content Based Recommendation

###  Dataset Used : TMDM

In [2]:
# importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [96]:
df = pd.read_csv('movie_dataset.csv')
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [6]:
# df shape
df.shape

(4803, 24)

In [89]:
df.columns





Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [101]:
## Dropping columns that are not revelent

df = df[['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity',
     'revenue', 'runtime',
         'title', 'vote_average',
       'vote_count', 'cast', 'director']]

In [103]:
## columns remaning after dropping non relevent columns

df.columns

Index(['budget', 'genres', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'revenue', 'runtime',
       'title', 'vote_average', 'vote_count', 'cast', 'director'],
      dtype='object')

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             4803 non-null   int64  
 1   genres             4775 non-null   object 
 2   id                 4803 non-null   int64  
 3   keywords           4391 non-null   object 
 4   original_language  4803 non-null   object 
 5   original_title     4803 non-null   object 
 6   overview           4799 non-null   object 
 7   popularity         4803 non-null   float64
 8   revenue            4803 non-null   int64  
 9   runtime            4801 non-null   float64
 10  title              4803 non-null   object 
 11  vote_average       4803 non-null   float64
 12  vote_count         4803 non-null   int64  
 13  cast               4760 non-null   object 
 14  director           4773 non-null   object 
dtypes: float64(3), int64(4), object(8)
memory usage: 563.0+ KB


In [105]:
df.describe()

,budget,id,popularity,revenue,runtime,vote_average,vote_count
count,4.803000e+03,4803.000000,4803.000000,4.803000e+03,4801.000000,4803.000000,4803.000000
mean,2.904504e+07,57165.484281,21.492301,8.226064e+07,106.875859,6.092172,690.217989
std,4.072239e+07,88694.614033,31.816650,1.628571e+08,22.611935,1.194612,1234.585891
min,0.000000e+00,5.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,7.900000e+05,9014.500000,4.668070,0.000000e+00,94.000000,5.600000,54.000000
50%,1.500000e+07,14629.000000,12.921594,1.917000e+07,103.000000,6.200000,235.000000
75%,4.000000e+07,58610.500000,28.313505,9.291719e+07,118.000000,6.800000,737.000000
max,3.800000e+08,459488.000000,875.581305,2.787965e+09,338.000000,10.000000,13752.000000


In [120]:
df_null=df[df.isnull().any(axis=1)]
df_null

,budget,genres,id,keywords,original_language,original_title,overview,popularity,revenue,runtime,title,vote_average,vote_count,cast,director
71,145000000,Adventure Action Fantasy,1735,NaN,en,The Mummy: Tomb of the Dragon Emperor,"Archaeologist Rick O'Connell travels to China,...",60.034162,401128639,112.0,The Mummy: Tomb of the Dragon Emperor,5.2,1387,Brendan Fraser Jet Li John Hannah Maria Bello ...,Rob Cohen
83,27000000,Action Adventure Science Fiction Romance,79698,NaN,en,The Lovers,The Lovers is an epic romance time travel adve...,2.418535,0,109.0,The Lovers,4.8,34,Josh Hartnett Simone Kessell Tamsin Egerton Al...,Roland Joff\u00e9
323,100000000,Comedy Drama Romance,37786,NaN,en,Sex and the City 2,"Carrie, Charlotte, and Miranda are all married...",18.325897,288347692,146.0,Sex and the City 2,5.4,426,Sarah Jessica Parker Kristin Davis Cynthia Nix...,Michael Patrick King
381,90000000,Fantasy Action Family,49852,NaN,en,The Nutcracker: The Untold Story,"Set in 1920's Vienna, this a tale of a little ...",3.593349,16178959,110.0,The Nutcracker: The Untold Story,5.4,50,Elle Fanning Nathan Lane John Turturro Frances...,Andrei Konchalovsky
436,80000000,Comedy,109418,NaN,en,Grown Ups 2,The all-star comedy cast from Grown Ups return...,45.589568,246984278,100.0,Grown Ups 2,5.8,1155,Adam Sandler Kevin James Chris Rock David Spad...,Dennis Dugan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4790,0,Drama Foreign,13898,NaN,fa,دایره,Various women struggle to function in the oppr...,1.193779,0,90.0,The Circle,6.6,17,Nargess Mamizadeh Maryiam Palvin Almani Mojgan...,Jafar Panahi
4794,0,Thriller Horror Comedy,286939,NaN,en,Sanctuary: Quite a Conundrum,"It should have been just a normal day of sex, ...",0.166513,0,82.0,Sanctuary: Quite a Conundrum,0.0,0,Sasha Ramos Erin Cline Emily Rogers Anthony Ru...,Thomas L. Phillips
4797,0,Foreign Thriller,67238,NaN,en,Cavite,"Adam, a security guard, travels from Californi...",0.022173,0,80.0,Cavite,7.5,2,NaN,Neill Dela Llana
4799,9000,Comedy Romance,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,0,85.0,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns


In [121]:
df_null.shape # 428 rows with null values

(428, 15)